<img src="https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png" style="float: left; margin: 15px;">

## Bagging and Boosting vs. Regression

Week 8 | 1.3

---

This lab uses the housing data from Project 3 to compare bagging and boosting ensemble methods to regression.

### 1. Load packages and data

In [1]:
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

sns.set_style('whitegrid')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

**You may want to load a cleaned/feature engineered version of the data that you have from project 3 rather than the raw housing data file (to skip the data munging and cleaning part.**

My path to the raw file is below; replace this with your own dataset.

In [2]:
house = pd.read_csv('/Users/kiefer/github-repos/DSI-SF-4/datasets/housing_regression/housing.csv')

In [4]:
processed = pd.read_csv('./train_test.csv')

In [5]:
print house.shape, processed.shape

(1460, 81) (2919, 146)


In [6]:
print processed.columns

Index([u'Intercept', u'BldgType[T.2fmCon]', u'BldgType[T.Duplex]',
       u'BldgType[T.Twnhs]', u'BldgType[T.TwnhsE]', u'CentralAir[T.Y]',
       u'ConditionOne[T.Feedr]', u'ConditionOne[T.Norm]',
       u'ConditionOne[T.PosA]', u'ConditionOne[T.PosN]',
       ...
       u'OverallQual', u'PoolArea', u'ScreenPorch', u'TotRmsAbvGrd',
       u'WoodDeckSF', u'YearBuilt', u'YearRemodAdd', u'YrAgeWhenSold',
       u'YrSold', u'is_test'],
      dtype='object', length=146)


In [8]:
print processed['is_test'].sum(), processed['is_test'].sum()+house.shape[0]

1459 2919


In [9]:
house.columns

Index([u'Id', u'MSSubClass', u'MSZoning', u'LotFrontage', u'LotArea',
       u'Street', u'Alley', u'LotShape', u'LandContour', u'Utilities',
       u'LotConfig', u'LandSlope', u'Neighborhood', u'Condition1',
       u'Condition2', u'BldgType', u'HouseStyle', u'OverallQual',
       u'OverallCond', u'YearBuilt', u'YearRemodAdd', u'RoofStyle',
       u'RoofMatl', u'Exterior1st', u'Exterior2nd', u'MasVnrType',
       u'MasVnrArea', u'ExterQual', u'ExterCond', u'Foundation', u'BsmtQual',
       u'BsmtCond', u'BsmtExposure', u'BsmtFinType1', u'BsmtFinSF1',
       u'BsmtFinType2', u'BsmtFinSF2', u'BsmtUnfSF', u'TotalBsmtSF',
       u'Heating', u'HeatingQC', u'CentralAir', u'Electrical', u'1stFlrSF',
       u'2ndFlrSF', u'LowQualFinSF', u'GrLivArea', u'BsmtFullBath',
       u'BsmtHalfBath', u'FullBath', u'HalfBath', u'BedroomAbvGr',
       u'KitchenAbvGr', u'KitchenQual', u'TotRmsAbvGrd', u'Functional',
       u'Fireplaces', u'FireplaceQu', u'GarageType', u'GarageYrBlt',
       u'GarageFinish',

In [10]:
output = house['SalePrice'].values
ln_output = np.log(output+1)

In [11]:
train, test = processed[processed['is_test'] == 0], processed[processed['is_test'] == 1]
print train.shape, test.shape

(1460, 146) (1459, 146)


In [12]:
train.drop('Intercept', axis=1, inplace=True)
test.drop('Intercept', axis=1, inplace=True)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [13]:
print train.shape, test.shape

(1460, 145) (1459, 145)


---

### 2. Build a decision tree regressor

1. Train a decision tree regressor on the regression problem (predicting `SalePrice` or a transformed version of it from predictors of your choice.)
- Evaluate the score with a 5-fold cross-validation
- How does this compare to the model you fit on this data for Project 3?


In [14]:
from sklearn.tree import DecisionTreeRegressor

In [15]:
y = ln_output
X = train

In [16]:
from sklearn.model_selection import GridSearchCV

In [18]:
dtc = DecisionTreeRegressor()
dtc_params = {
    'max_depth':[2,3,4,5,6,7,8,None],
    'min_samples_split':[2,4,8,16,32,64,128],
    'max_features':[None, 'sqrt', 'log2']
}

dtc_gs = GridSearchCV(dtc, dtc_params, cv=5, verbose=1)
dtc_gs.fit(X, y)

Fitting 5 folds for each of 168 candidates, totalling 840 fits


[Parallel(n_jobs=1)]: Done 840 out of 840 | elapsed:    7.8s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best'),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'min_samples_split': [2, 4, 8, 16, 32, 64, 128], 'max_features': [None, 'sqrt', 'log2'], 'max_depth': [2, 3, 4, 5, 6, 7, 8, None]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=1)

In [19]:
print dtc_gs.best_params_
dtc_best = dtc_gs.best_estimator_

{'max_features': None, 'min_samples_split': 32, 'max_depth': None}


In [20]:
from sklearn.model_selection import cross_val_score

In [21]:
dtc_scores = cross_val_score(DecisionTreeRegressor(**dtc_gs.best_params_), X, y, cv=5)
print dtc_scores
print np.mean(dtc_scores), np.std(dtc_scores)

[ 0.73845805  0.73312997  0.79097556  0.76427937  0.72259966]
0.749888523227 0.0247039596558


---

### 3. Build a random forest regressor

1. Train a random forest regressor on the regression problem.
- Evaluate the score with a 5-fold cross-validation
- How does this compare to the models you fit on this data previously?

You may want to use a gridsearch to find the optimal parameters. Be careful to not put too many different options/parameters into the gridsearch or it will take a long time to fit.

In [22]:
from sklearn.ensemble import RandomForestRegressor

In [26]:
rf = RandomForestRegressor()

rf_params = {
    'n_estimators':[200],
    'max_depth':[2,3,4,5,6,None],
    'min_samples_split':[2,4,8,16,32,64,128],
    'max_features':[None, 'sqrt', 'log2']
}

rf_gs = GridSearchCV(rf, rf_params, cv=5, verbose=1, n_jobs=-1)
rf_gs.fit(X, y)

Fitting 5 folds for each of 126 candidates, totalling 630 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   25.3s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 630 out of 630 | elapsed:  5.9min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [200], 'min_samples_split': [2, 4, 8, 16, 32, 64, 128], 'max_depth': [2, 3, 4, 5, 6, None], 'max_features': [None, 'sqrt', 'log2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=1)

In [27]:
print rf_gs.best_params_
rf_best = rf_gs.best_estimator_

rf_scores = cross_val_score(rf_best, X, y, cv=5)
print rf_scores
print np.mean(rf_scores), np.std(rf_scores)

{'max_features': 'sqrt', 'min_samples_split': 2, 'n_estimators': 200, 'max_depth': None}
[ 0.8722039   0.8578691   0.85176007  0.86764115  0.84539344]
0.858973531787 0.00987768151303


---

### 4. Build an AdaBoost regressor and/or a gradient boosted regressor

The models both allow you to change the base estimator (they default to decision tree regressors, which I would recommend). The most important parameters in Adaboost besides the `base_estimator` for each model are:

    n_estimators: how many weak learners to chain together
    learning_rate: how much should the contribution of subsequent weak learners be "shrunk" (this means that in addition to the reweighting, subsequent weak learners are also forced to have a smaller impact.)
    
The gradient boosting regressor forces you to use decision trees, but allows you to modify components of each weak learner tree through its keyword arguments, such as `max_depth`, `max_features`, `min_samples_split`, etc. 

1. Build the model(s). You may want to find the best parameters with a gridsearch.
2. Evaluate the score using cross-validation as before.
3. How does boosting compare to bagging (random forest) and the original model you built for your project?

In [28]:
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor

In [30]:
ada = AdaBoostRegressor()

ada_params = {
    'n_estimators':[25,50,100,150,200],
    'learning_rate':[0.5,0.1,0.05,0.01,0.001],
}

ada_gs = GridSearchCV(ada, ada_params, cv=5, verbose=1, n_jobs=-1)
ada_gs.fit(X, y)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   15.2s
[Parallel(n_jobs=-1)]: Done 125 out of 125 | elapsed:   51.0s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='linear',
         n_estimators=50, random_state=None),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [25, 50, 100, 150, 200], 'learning_rate': [0.5, 0.1, 0.05, 0.01, 0.001]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=1)

In [31]:
print ada_gs.best_params_
ada_best = ada_gs.best_estimator_

ada_scores = cross_val_score(ada_best, X, y, cv=5)
print ada_scores
print np.mean(ada_scores), np.std(ada_scores)

{'n_estimators': 150, 'learning_rate': 0.5}
[ 0.75730924  0.82841149  0.7944712   0.7745997   0.7750997 ]
0.785978265801 0.0242587567211


In [32]:
gb = GradientBoostingRegressor()

gb_params = {
    'n_estimators':[200],
    'loss':['ls','huber'],
    'learning_rate':[0.5,0.25,0.1,0.01],
    'max_depth':[2,3,4,5,6,None],
    'min_samples_split':[2,16,64,128],
    'max_features':[None, 'sqrt', 'log2'],
    'subsample':[1.0,0.75,0.5]
}

gb_gs = GridSearchCV(gb, gb_params, cv=5, verbose=1, n_jobs=-1)
gb_gs.fit(X, y)

Fitting 5 folds for each of 1728 candidates, totalling 8640 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    9.2s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   23.9s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed:  7.6min
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed: 11.7min
[Parallel(n_jobs=-1)]: Done 3192 tasks      | elapsed: 15.8min
[Parallel(n_jobs=-1)]: Done 4042 tasks      | elapsed: 19.8min
[Parallel(n_jobs=-1)]: Done 4992 tasks      | elapsed: 24.5min
[Parallel(n_jobs=-1)]: Done 6042 tasks      | elapsed: 31.3min
[Parallel(n_jobs=-1)]: Done 7192 tasks      | elapsed: 39.4min
[Parallel(n_jobs=-1)]: Done 8442 tasks      | elapsed: 48.0min
[Parallel(n_jobs=-1)]: Done 8640 out of 8640 | elapsed: 53.0min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=100,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'loss': ['ls', 'huber'], 'subsample': [1.0, 0.75, 0.5], 'learning_rate': [0.5, 0.25, 0.1, 0.01], 'n_estimators': [200], 'max_features': [None, 'sqrt', 'log2'], 'min_samples_split': [2, 16, 64, 128], 'max_depth': [2, 3, 4, 5, 6, None]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=1)

In [33]:
print gb_gs.best_params_
gb_best = gb_gs.best_estimator_

gb_scores = cross_val_score(gb_best, X, y, cv=5)
print gb_scores
print np.mean(gb_scores), np.std(gb_scores)

{'loss': 'huber', 'subsample': 0.5, 'learning_rate': 0.1, 'n_estimators': 200, 'max_features': None, 'min_samples_split': 2, 'max_depth': 3}
[ 0.89694798  0.8849864   0.87471605  0.8874101   0.86502796]
0.881817698945 0.0109778850747


---

### 5. [Bonus] Submit to kaggle with your bagging and/or boosting model.

Using the test data like we did in class, make predictions and submit your score to kaggle. Does your ensemble model perform better?

I've put the path to my test data below (which should be in the equivalent folder in your repo).

In [6]:
test = pd.read_csv('/Users/kiefer/github-repos/DSI-SF-4/datasets/housing_regression/test_houses.csv')
